In [1]:
import pandas as pd
import ast
import re

df = pd.read_csv('data/train+klue.csv')

In [2]:
# problems 열 문자열을 딕셔너리로 변환
df['problems'] = df['problems'].apply(ast.literal_eval)

# answer 값 추출 및 개수 세기
answer_counts = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts)

problems
1    948
2    450
3    319
4    283
5     31
Name: count, dtype: int64


In [3]:
# choices 길이 계산 및 분포 확인
df['choices_length'] = df['problems'].apply(lambda x: len(x['choices']))

# 선택지 길이별 분포 확인
choices_length_distribution = df['choices_length'].value_counts()

# 결과 출력
print(choices_length_distribution)


choices_length
5    1239
4     792
Name: count, dtype: int64


In [4]:
# 948 - 541 = 407
# 450 - 44 = 406
# 319 + 43 + 44 = 406
# 283 + 123 = 406
# 31 + 375 = 406

In [5]:
df_1 = df[df['problems'].apply(lambda x: x['answer'] == 1)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_1 = df_1.sort_index()

# 선택지 길이가 5인 행만 필터링
df_choices_length_5 = df_1[df_1['choices_length'] == 5]

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_1to5 = df_choices_length_5.sample(n=375, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    if len(choices) >= 5:  # 다섯 번째 선택지가 존재할 경우에만 처리
        choices[0], choices[4] = choices[4], choices[0]
    problems['choices'] = choices
    return problems

sampled_rows_1to5['problems'] = sampled_rows_1to5['problems'].apply(swap_choices)

# answer 값을 5로 변경하는 함수
def set_answer_to_5(problems):
    problems['answer'] = 5  # answer 값을 5로 설정
    return problems

# sampled_rows에서 answer 값을 5로 변경
sampled_rows_1to5['problems'] = sampled_rows_1to5['problems'].apply(set_answer_to_5)

def replace_klue_last_digit(klue, new_digit):
    return re.sub(r'\d+$', str(new_digit), klue)

sampled_rows_1to5['klue'] = sampled_rows_1to5['klue'].apply(lambda x: replace_klue_last_digit(x, 5))

sampled_rows_1to5 = sampled_rows_1to5.drop(columns=['choices_length'])

print(sampled_rows_1to5.head())

                          id  \
1384  yongari_generate_67647   
1217  yongari_generate_40560   
2002  yongari_generate_10822   
1110  yongari_generate_61003   
1336  yongari_generate_59803   

                                              paragraph  \
1384  사회적 거리두기가 일상화되면서 사무실, 공장, 음식점 등 다중이용시설에서 ‘아크릴 ...   
1217  지난해 3차 양적완화의 필요성을 강하게 주장했던 미국 중앙은행(Fed)의 대표적 비...   
2002  1995년 외식체인점 후터스의 미국 플로리다 잭슨빌 매장에 17세의 갈색머리 소녀가...   
1110  허니비즈(대표 윤문진)는 서울시 ‘서울사랑상품권’에 이어, 경기도 시흥시와 지역화폐...   
1336  핀란드 관광청이 3일 온라인 브리핑을 통해 지속가능한 핀란드 여행(Sustainab...   

                                               problems  \
1384  {'question': '애드피아몰이 출시한 아크릴 가림막의 주된 용도는 무엇인가?...   
1217  {'question': '윌리엄스 총재가 3차 양적완화의 속도 조절 가능성을 제기한...   
2002  {'question': '카트리나 콜이 후터스에서 일하기 시작한 이유는 무엇인가?'...   
1110  {'question': '허니비즈가 시흥시와 체결한 협약의 주요 목적은 무엇인가?'...   
1336  {'question': '핀란드 관광청이 공식 론칭한 프로그램의 이름은 무엇인가?'...   

                                                   klue  question_plus  
1384  근거 : 내용에 따르면, 애드피아

In [6]:
df_1to5 = sampled_rows_1to5[sampled_rows_1to5['problems'].apply(lambda x: x['answer'] == 5)]
df_1to5

,id,paragraph,problems,klue,question_plus
1384,yongari_generate_67647,"사회적 거리두기가 일상화되면서 사무실, 공장, 음식점 등 다중이용시설에서 ‘아크릴 ...",{'question': '애드피아몰이 출시한 아크릴 가림막의 주된 용도는 무엇인가?...,"근거 : 내용에 따르면, 애드피아몰이 출시한 아크릴 가림막은 비말 확산을 방지하기 ...",NaN
1217,yongari_generate_40560,지난해 3차 양적완화의 필요성을 강하게 주장했던 미국 중앙은행(Fed)의 대표적 비...,{'question': '윌리엄스 총재가 3차 양적완화의 속도 조절 가능성을 제기한...,"근거 : 윌리엄스 총재는 '고용 시장 전망의 상당한 개선'이 기대되고 있으며, 미국...",NaN
2002,yongari_generate_10822,1995년 외식체인점 후터스의 미국 플로리다 잭슨빌 매장에 17세의 갈색머리 소녀가...,{'question': '카트리나 콜이 후터스에서 일하기 시작한 이유는 무엇인가?'...,근거 : 카트리나 콜은 홀어머니를 도와 경제적 형편을 개선하기 위해 후터스에서 일하...,NaN
1110,yongari_generate_61003,"허니비즈(대표 윤문진)는 서울시 ‘서울사랑상품권’에 이어, 경기도 시흥시와 지역화폐...",{'question': '허니비즈가 시흥시와 체결한 협약의 주요 목적은 무엇인가?'...,근거 : 허니비즈와 시흥시가 체결한 협약의 주요 목적은 기존 배달주문 시장의 문제점...,NaN
1336,yongari_generate_59803,핀란드 관광청이 3일 온라인 브리핑을 통해 지속가능한 핀란드 여행(Sustainab...,{'question': '핀란드 관광청이 공식 론칭한 프로그램의 이름은 무엇인가?'...,근거 : 핀란드 관광청은 온라인 브리핑을 통해 ‘지속가능한 핀란드 여행(Sustai...,NaN
...,...,...,...,...,...
2029,yongari_generate_73938,방하남 고용노동부 장관(사진)이 취임 후 첫 외부 행사로 5년 만에 일자리를 3.5...,{'question': '방하남 고용노동부 장관이 방문한 기업의 이름은 무엇인가?'...,근거 : 내용에 따르면 방하남 고용노동부 장관은 인천 송도에 위치한 셀트리온을 방문...,NaN
1824,yongari_generate_82524,"지역별로는 수도권이 1월보다 3419가구 줄어든 2만9278가구를 기록, 4개월 연...","{'question': '미분양 주택이 줄어든 주된 이유는 무엇인가?', 'choi...",근거 : 미분양 주택이 줄어든 주된 이유는 부동산 규제 완화와 전세 수요의 매매 전...,NaN
991,yongari_generate_37230,"한우산업의 안정적 발전을 위해 설립된 한우자조금관리위원회(위원장 민경천, 이하 한우...",{'question': '2020 한우영상공모전에서 1등을 수상한 작품의 제목은 무...,"근거 : 내용에 따르면, 2020 한우영상공모전에서 1등으로 선정된 작품의 제목은 ...",NaN
901,yongari_generate_22197,"㈜엔씨소프트(대표 김택진, 이하 엔씨(NC))는 고용노동부에서 주관하는 ‘2020 ...",{'question': '㈜엔씨소프트가 2020 대한민국 일자리 으뜸기업으로 선정된...,"근거 : ㈜엔씨소프트는 일자리 증가와 질 개선에 앞장서며, 직원 평균 연령이 낮고 ...",NaN


In [7]:
df = df.drop(columns=['choices_length'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   klue           2031 non-null   object 
 4   question_plus  0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 79.5+ KB


In [8]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_1to5.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_5 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_5)

problems
1    573
2    450
3    319
4    283
5     31
Name: count, dtype: int64


In [9]:
df_1 = df[df['problems'].apply(lambda x: x['answer'] == 1)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_1 = df_1.sort_index()

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_1to4 = df_1.sample(n=123, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    choices[0], choices[3] = choices[3], choices[0]
    problems['choices'] = choices
    return problems

sampled_rows_1to4['problems'] = sampled_rows_1to4['problems'].apply(swap_choices)

# answer 값을 5로 변경하는 함수
def set_answer_to_4(problems):
    problems['answer'] = 4  # answer 값을 5로 설정
    return problems

# sampled_rows에서 answer 값을 5로 변경
sampled_rows_1to4['problems'] = sampled_rows_1to4['problems'].apply(set_answer_to_4)

def replace_klue_last_digit(klue, new_digit):
    return re.sub(r'\d+$', str(new_digit), klue)

sampled_rows_1to4['klue'] = sampled_rows_1to4['klue'].apply(lambda x: replace_klue_last_digit(x, 4))

print(sampled_rows_1to4.head())

                          id  \
1839  yongari_generate_21675   
312   yongari_generate_18390   
601   yongari_generate_86382   
1535  yongari_generate_74992   
1943  yongari_generate_28492   

                                              paragraph  \
1839  푸마(PUMA)가 지난 12일(한국시간), 전 세계 축구 슈퍼스타 중 한 명인 ‘네...   
312   낸시는 1시간 동안 시험 공부를 하거나, 6달러의 임금을 받고 1시간 동안 잔디를 ...   
601   이 질문은 다음 정보를 참조한다.\n그는 나를 붙잡고 어둠의 집으로 끌고갔다…\n한...   
1535  “노조 파업으로 자동차 생산라인이 멈춰서고, 하청업체는 일감 부족으로 고통을 받는 ...   
1943  극심한 청년 취업난 속 취업준비생들은 상반기 공채시즌에 집중하고 있다. 또한 회계사...   

                                               problems  \
1839  {'question': '네이마르가 푸마와 체결한 파트너십의 주요 내용은 무엇인가?...   
312   {'question': '낸시가 시험 공부를 하는 것을 선택했다는 것을 알고 있다면...   
601   {'question': '다음 중 위 두 개의 글이 가장 잘 뒷받침하는 결론은 무엇...   
1535  {'question': '현대차 노조의 최근 파업 결정에 대한 반발을 나타낸 인물은...   
1943  {'question': 'KG에듀원 미래경영아카데미가 실시하는 모의고사의 주요 목적...   

                                                   klue  question_plus  
1839  근거 : 네이마르가 푸마와 체결한

In [10]:
df_1to4 = sampled_rows_1to4[sampled_rows_1to4['problems'].apply(lambda x: x['answer'] == 4)]
df_1to4

,id,paragraph,problems,klue,question_plus
1839,yongari_generate_21675,"푸마(PUMA)가 지난 12일(한국시간), 전 세계 축구 슈퍼스타 중 한 명인 ‘네...",{'question': '네이마르가 푸마와 체결한 파트너십의 주요 내용은 무엇인가?...,근거 : 네이마르가 푸마와 체결한 파트너십의 주요 내용은 푸마의 대표 축구화인 ‘푸...,NaN
312,yongari_generate_18390,"낸시는 1시간 동안 시험 공부를 하거나, 6달러의 임금을 받고 1시간 동안 잔디를 ...",{'question': '낸시가 시험 공부를 하는 것을 선택했다는 것을 알고 있다면...,"근거 : 내용에 따르면, 낸시는 임금이 없는 시험 공부를 선택함으로써 6달러의 잔디...",NaN
601,yongari_generate_86382,이 질문은 다음 정보를 참조한다.\n그는 나를 붙잡고 어둠의 집으로 끌고갔다…\n한...,{'question': '다음 중 위 두 개의 글이 가장 잘 뒷받침하는 결론은 무엇...,근거 : 내용에서 어둠의 집과 진리의 길은 상반된 개념으로 제시된다. 첫 번째 글은...,NaN
1535,yongari_generate_74992,"“노조 파업으로 자동차 생산라인이 멈춰서고, 하청업체는 일감 부족으로 고통을 받는 ...",{'question': '현대차 노조의 최근 파업 결정에 대한 반발을 나타낸 인물은...,"근거 : 내용 형식으로 보자면, 반발의 목소리를 낸 인물은 현대차 협력업체의 사장과...",NaN
1943,yongari_generate_28492,극심한 청년 취업난 속 취업준비생들은 상반기 공채시즌에 집중하고 있다. 또한 회계사...,{'question': 'KG에듀원 미래경영아카데미가 실시하는 모의고사의 주요 목적...,근거 : KG에듀원 미래경영아카데미가 실시하는 모의고사의 주요 목적은 수험생들이 시...,NaN
...,...,...,...,...,...
271,yongari_generate_25803,정부가 방금 개인 소득세를 인하했습니다.,{'question': '다음 중 이 정책이 경제에 미치는 영향에 대해 가장 잘 설...,"근거 : 내용에 따르면, 개인 소득세 인하는 가처분 소득을 증가시켜 소비를 촉진하고...",NaN
86,yongari_generate_16217,먼저 우주가 구형이라는 점에 주목해야 한다. 부분적으로는 연결점이 필요 없는 완전한...,{'question': '위 글을 보았을 때 구체 형태의 우주에 관한 코페르니쿠스의...,"근거 : 코페르니쿠스는 우주가 구형이라는 점을 강조하며, 이는 자연의 본질과 천체들...",NaN
1798,yongari_generate_33423,서울중앙지검 금융조세조사2부(부장검사 김범기)는 30일 가전업체 모뉴엘로부터 금품을...,{'question': '조계륭 전 한국무역보험공사 사장이 구속영장이 청구된 이유는...,근거 : 조계륭 전 한국무역보험공사 사장은 금품을 수수한 혐의(뇌물)로 구속영장이 ...,NaN
620,yongari_generate_71635,프랑스 혁명의 위대한 순수성이야말로 우리의 약점과 강점 모두를 끌어내는 것입니다. ...,"{'question': '위 글의 화자가 가장 염려하고 있는 위협은 무엇입니까?',...","근거 : 화자는 ""모든 악한 이들이 우리를 적대시하게 된다""는 점에서 외부 적의 위...",NaN


In [11]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_1to4.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_4 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_4)

problems
2    450
1    450
3    319
4    283
5     31
Name: count, dtype: int64


In [12]:
df_1 = df[df['problems'].apply(lambda x: x['answer'] == 1)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_1 = df_1.sort_index()

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_1to3 = df_1.sample(n=43, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    choices[0], choices[2] = choices[2], choices[0]
    problems['choices'] = choices
    return problems

sampled_rows_1to3['problems'] = sampled_rows_1to3['problems'].apply(swap_choices)

# answer 값을 3으로 변경하는 함수
def set_answer_to_3(problems):
    problems['answer'] = 3  # answer 값을 3로 설정
    return problems

# sampled_rows에서 answer 값을 3로 변경
sampled_rows_1to3['problems'] = sampled_rows_1to3['problems'].apply(set_answer_to_3)

def replace_klue_last_digit(klue, new_digit):
    return re.sub(r'\d+$', str(new_digit), klue)

sampled_rows_1to3['klue'] = sampled_rows_1to3['klue'].apply(lambda x: replace_klue_last_digit(x, 3))

print(sampled_rows_1to3.head())

                          id  \
1847  yongari_generate_46440   
1996  yongari_generate_86051   
691   yongari_generate_69506   
172   yongari_generate_87682   
1877  yongari_generate_42568   

                                              paragraph  \
1847  전국 최대 검찰청인 서울중앙지검은 13일 차장·부장검사가 중요 사건을 직접 수사하는...   
1996  미국에 이어 스페인에서도 에볼라 바이러스에 감염된 환자가 확인됐다.월스트리트저널(W...   
691   린 쩌쉬, 중국 무역 대표, 빅토리아 여왕에게 보낸 편지, 1839. 그렇다면 어떤...   
172   이 지문에 나타난 정보는 무엇입니까? 다음 편지를 읽어보십시오. 사건은 . . . ...   
1877  정부가 손자병법까지 인용하며 외환시장의 불안심리를 선제적으로 차단하고 투기세력은 강...   

                                               problems  \
1847  {'question': '서울중앙지검의 수사 개선안의 주된 목적은 무엇인가?', '...   
1996  {'question': '스페인에서 에볼라 바이러스에 감염된 간호사가 치료한 환자의...   
691   {'question': '다음 중 린 쩌쉬의 편지가 전달된 후 뒤따른 결과는?', ...   
172   {'question': '다음 중 17세기 말 실현된 유럽의 아메리카 정복의 결과가...   
1877  {'question': '추경호 기획재정부 제1차관이 인용한 손자병법의 문장은 어떤...   

                                                   klue  question_plus  
1847  근거 : 서울중앙지검의 수사 개선

In [13]:
df_1to3 = sampled_rows_1to3[sampled_rows_1to3['problems'].apply(lambda x: x['answer'] == 3)]
len(df_1to3)

43

In [14]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_1to3.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_3 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_3)

problems
2    450
1    407
3    319
4    283
5     31
Name: count, dtype: int64


In [15]:
df_2 = df[df['problems'].apply(lambda x: x['answer'] == 2)]

# 인덱스를 정렬하여 항상 같은 순서를 보장
df_2 = df_2.sort_index()

# 375개 행을 랜덤하게 선택 (고정된 시드값을 사용하여 항상 동일한 샘플링)
sampled_rows_2to3 = df_2.sample(n=44, random_state=42)

# choices 리스트의 첫 번째와 다섯 번째 선택지 위치 변경
def swap_choices(problems):
    choices = problems['choices']
    choices[1], choices[2] = choices[2], choices[1]
    problems['choices'] = choices
    return problems

sampled_rows_2to3['problems'] = sampled_rows_2to3['problems'].apply(swap_choices)

# answer 값을 3으로 변경하는 함수
def set_answer_to_3(problems):
    problems['answer'] = 3  # answer 값을 3로 설정
    return problems

# sampled_rows에서 answer 값을 3로 변경
sampled_rows_2to3['problems'] = sampled_rows_2to3['problems'].apply(set_answer_to_3)

def replace_klue_last_digit(klue, new_digit):
    return re.sub(r'\d+$', str(new_digit), klue)

sampled_rows_2to3['klue'] = sampled_rows_2to3['klue'].apply(lambda x: replace_klue_last_digit(x, 3))

print(sampled_rows_2to3.head())

                          id  \
1813  yongari_generate_68435   
1994  yongari_generate_18441   
522   yongari_generate_43156   
129   yongari_generate_37474   
1849  yongari_generate_54389   

                                              paragraph  \
1813  한국이 쌀 시장 개방을 미루려면 이해 당사국들에 보상을 해야 한다고 호베르토 아제베...   
1994  공급 과잉 논란 속에 배후 임대수요 확보 여부가 ‘수익형 부동산’인 오피스텔 선택 ...   
522   종교적 직업과 예배에서 양심의 자유가 제한된다면 시민의 자유를 최대한 누리더라도 진...   
129   "중국 황제 폐하께서는 영국 백성이 가족 및 시설과 함께 상업을 수행할 목적으로 박...   
1849  애플이 사상 처음으로 시가총액 7000억달러(약 774조원) 고지를 찍으면서 미국 ...   

                                               problems  \
1813  {'question': '호베르토 아제베도 WTO 사무총장이 언급한 한국의 쌀 관세...   
1994  {'question': '오피스텔 투자에서 안정적인 임대수요를 확보하기 위해 가장 ...   
522   {'question': '다음 중 펜실베이니아 식민지를 가장 잘 설명한 것은 무엇입...   
129   {'question': '이 조약에서 제안된 양해가 다음 중 영국의 무엇을 직접적으...   
1849  {'question': '애플의 시가총액이 7000억 달러에 도달한 후, 주가 상승...   

                                                   klue  question_plus  
1813  근거 : 아제베도 WTO 사무총장

In [16]:
df_2to3 = sampled_rows_2to3[sampled_rows_2to3['problems'].apply(lambda x: x['answer'] == 3)]
len(df_2to3)

44

In [17]:
# sampled_rows의 인덱스를 가져와서 df에서 해당 인덱스를 제거
df = df.drop(sampled_rows_2to3.index)

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_4 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_4)

problems
1    407
2    406
3    319
4    283
5     31
Name: count, dtype: int64


In [18]:
# 여러 데이터프레임 합치기
merged_sampled_rows = pd.concat([sampled_rows_1to5, sampled_rows_1to4, sampled_rows_1to3, sampled_rows_2to3])

# df와 합치기
df = pd.concat([df, merged_sampled_rows])

# 결과 확인
# answer 값 추출 및 개수 세기
answer_counts_5 = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts_5)

problems
1    407
2    406
4    406
3    406
5    406
Name: count, dtype: int64


In [20]:
# id에서 숫자 부분을 추출하여 정렬 기준으로 사용
df['id_number'] = df['id'].apply(lambda x: int(x.split('_')[-1]))

# id_number 열을 기준으로 오름차순 정렬
df = df.sort_values(by='id_number')

# 불필요한 'id_number' 열을 삭제
df = df.drop(columns=['id_number'])

# 결과 확인
print(df.head())

                          id  \
7     yongari_generate_10019   
1505  yongari_generate_10114   
1313  yongari_generate_10148   
702   yongari_generate_10170   
1951  yongari_generate_10200   

                                              paragraph  \
7     신돈이 (가)을/를 설치하자고 요청하자, …(중략)…이 제 도감이 설치되었다. …(...   
1505  중국 정부가 과잉 생산해 온 19개 분야 1400여개 기업에 대한 규제에 나섰다. ...   
1313  ‘왓츠앱’(사진) ‘아이메시지’ 등 대화용 애플리케이션(앱·응용프로그램)이 빠르게 ...   
702   “푸루사-수크타” 푸루사는 불멸의 군주이며 [제례] 음식으로 자란다. 신들이 푸루사...   
1951  화재·붕괴·폭발·침수…. 블록버스터 영화 속에는 종종 다양한 재난 장면이 등장한다....   

                                               problems  \
7     {'question': '(가)에 대한 설명으로 옳은 것은?', 'choices':...   
1505  {'question': '중국 정부가 과잉 생산을 중단하라고 지시한 기업의 수는 몇...   
1313  {'question': '대화용 애플리케이션의 성장으로 인해 어떤 시장이 잠식되고 ...   
702   {'question': '다음 중 위 구절에서 설명하고 있는 힌두교의 요소는?', ...   
1951  {'question': '메모리얼 메디컬센터에서 발생한 재난 대응 실패의 주된 원인...   

                                                   klue  question_plus  
7     근거 : 내용에 따르면 신돈의 요

In [21]:
# problems 열을 문자열로 변환
df['problems'] = df['problems'].apply(str)

# 결과 확인
print(df['problems'].head())  # 변환된 problems 열 확인


7       {'question': '(가)에 대한 설명으로 옳은 것은?', 'choices':...
1505    {'question': '중국 정부가 과잉 생산을 중단하라고 지시한 기업의 수는 몇...
1313    {'question': '대화용 애플리케이션의 성장으로 인해 어떤 시장이 잠식되고 ...
702     {'question': '다음 중 위 구절에서 설명하고 있는 힌두교의 요소는?', ...
1951    {'question': '메모리얼 메디컬센터에서 발생한 재난 대응 실패의 주된 원인...
Name: problems, dtype: object


In [22]:
df.to_csv('data/train+klue_balanced.csv', index=False, encoding='utf-8')

In [23]:
train_balanced = pd.read_csv('data/train+klue_balanced.csv')

# problems 열 문자열을 딕셔너리로 변환
df['problems'] = df['problems'].apply(ast.literal_eval)

# answer 값 추출 및 개수 세기
answer_counts = df['problems'].apply(lambda x: x['answer']).value_counts()

# 결과 출력
print(answer_counts)

problems
1    407
3    406
5    406
2    406
4    406
Name: count, dtype: int64


In [24]:
train_balanced

,id,paragraph,problems,klue,question_plus
0,yongari_generate_10019,"신돈이 (가)을/를 설치하자고 요청하자, …(중략)…이 제 도감이 설치되었다. …(...","{'question': '(가)에 대한 설명으로 옳은 것은?', 'choices':...","근거 : 내용에 따르면 신돈의 요청으로 제 도감이 설치되었고, 이에 따라 전민이 주...",NaN
1,yongari_generate_10114,중국 정부가 과잉 생산해 온 19개 분야 1400여개 기업에 대한 규제에 나섰다. ...,{'question': '중국 정부가 과잉 생산을 중단하라고 지시한 기업의 수는 몇...,"근거 : 내용에 따르면, 중국 정부는 과잉 생산해 온 19개 분야에 1400여 개 ...",NaN
2,yongari_generate_10148,‘왓츠앱’(사진) ‘아이메시지’ 등 대화용 애플리케이션(앱·응용프로그램)이 빠르게 ...,{'question': '대화용 애플리케이션의 성장으로 인해 어떤 시장이 잠식되고 ...,근거 : 내용에 따르면 대화용 애플리케이션의 성장은 이동통신업계의 문자메시지 시장을...,NaN
3,yongari_generate_10170,“푸루사-수크타” 푸루사는 불멸의 군주이며 [제례] 음식으로 자란다. 신들이 푸루사...,"{'question': '다음 중 위 구절에서 설명하고 있는 힌두교의 요소는?', ...","근거 : 내용에 따르면, 푸루사는 제물로 희생되어 다양한 존재들이 태어나게 되었고,...",NaN
4,yongari_generate_10200,화재·붕괴·폭발·침수…. 블록버스터 영화 속에는 종종 다양한 재난 장면이 등장한다....,{'question': '메모리얼 메디컬센터에서 발생한 재난 대응 실패의 주된 원인...,근거 : 내용에 따르면 메모리얼 메디컬센터의 재난 대응 실패는 의사들이 상상하지 못...,NaN
...,...,...,...,...,...
2026,yongari_generate_99375,"오늘 밤, 아이들의 미래를 가장 큰 목표로 삼은 한 여성의 딸이 우리 모두를 위한 ...",{'question': '제랄딘 페라로의 부회장 지명과 가장 직접적으로 이어지는 일...,"근거 : 내용에 따르면, 오늘 밤 이탈리아 출신 이민자의 딸로서 부통령 출마자 지명...",NaN
2027,yongari_generate_99375,"오늘 밤, 아이들의 미래를 가장 큰 목표로 삼은 한 여성의 딸이 우리 모두를 위한 ...","{'question': '제랄딘 페라로와 가장 직접적으로 비견되는 여성은?', 'c...","근거 : 본문에서 ""아이들의 미래를 가장 큰 목표로 삼은 한 여성의 딸""이라는 표현...",NaN
2028,yongari_generate_99375,"오늘 밤, 아이들의 미래를 가장 큰 목표로 삼은 한 여성의 딸이 우리 모두를 위한 ...",{'question': '위의 구절에서 페라로가 표현한 정치사상이 가장 직접적으로 ...,근거 : 이 구절에서는 모든 미국인이 평등한 규칙 아래에서 살아야 한다는 주장을 통...,NaN
2029,yongari_generate_99832,총 공급 곡선이 왼쪽으로 이동하면 균형 물가 수준과 균형 산출량은 어떻게 되겠습니까...,{'question': '총 공급 곡선이 왼쪽으로 이동하면 균형 물가 수준과 균형 ...,근거 : 총 공급 곡선이 왼쪽으로 이동하면 공급이 줄어들어 균형 물가 수준은 상승하...,NaN
